# Notes to self:
Variables:
* Unprocessed or denoised
* Length of STFT window
* First second of all data or first sample or random second or all samples

In [ ]:
# Setup
import librosa
import os
%matplotlib inline
import matplotlib.pyplot as plt


def firstSecond(wav_names, prefix):
    """Grabs the first second of the first wav file."""
    w = next(wav_names)
    path = os.path.join(prefix, w)
    sample, sampling_rate = librosa.load(path)
    segment = sample[:sampling_rate]    # Grab a 1s segment
    return segment, sampling_rate

def firstSample(wav_names, prefix):
    """Grabs all of the first sample."""
    w = next(wav_names)
    path = os.path.join(prefix, w)
    return librosa.load(path)

notHidden = lambda x: not x.startswith('.')
def visualize(visFun, choiceFun=firstSecond, data_dir='../data'):
    classes = filter(notHidden, os.listdir(data_dir)) 
    for c in classes:
        wavs = filter(notHidden, os.listdir(data_dir + '/' + c))
        segment, sampling_rate = choiceFun(wavs, os.path.join(data_dir, c))
        visFun(c, segment, sampling_rate)



In [ ]:
from IPython.display import Audio
from IPython.display import display

def playAudio(label, segment, sampling_rate):
    print(label)
    display(Audio(segment, rate=sampling_rate, autoplay=False))

visualize(playAudio)

In [ ]:
def plotAmplitude(label, segment, sampling_rate):
    fig = plt.figure(figsize=(14,5))
    librosa.display.waveshow(segment, sr=sampling_rate, color='blue')
    fig.suptitle(label)

visualize(plotAmplitude)

In [ ]:
def viewSpectrogram(label, segment, sampling_rate):
    sgram = librosa.stft(segment, n_fft=16384)
    sgram_mag, _ = librosa.magphase(sgram)
    mel_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sampling_rate)

    fig = plt.figure(figsize=(14,5))
    librosa.display.specshow(mel_sgram)
    fig.suptitle(label)

visualize(viewSpectrogram)

In [ ]:
visualize(viewSpectrogram, choiceFun=firstSample)

In [ ]:
from IPython.display import Audio
from IPython.display import display
import librosa
import os


DATA_DIR = '../data'
notHidden = lambda x: not x.startswith('.')
classes = filter(notHidden, os.listdir(DATA_DIR))

# Play the first second of the first sample of each class
for c in classes:
    wavs = filter(notHidden, os.listdir(DATA_DIR + '/' + c))
    w = next(wavs)
    path = os.path.join(DATA_DIR, c, w)
    sample, sampling_rate = librosa.load(path)
    segment = sample[:sampling_rate]    # Grab a 1s segment
    print(c)
    display(Audio(segment, rate=sampling_rate, autoplay=False))


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import os

DATA_DIR = '../data'

notHidden = lambda x: not x.startswith('.')
classes = filter(notHidden, os.listdir(DATA_DIR))

# Play the first second of the first sample of each class
for c in classes:
    wavs = filter(notHidden, os.listdir(DATA_DIR + '/' + c))
    w = next(wavs)
    path = os.path.join(DATA_DIR, c, w)
    sample, sampling_rate = librosa.load(path)
    segment = sample[:sampling_rate]    # Grab a 1s segment
    fig = plt.figure(figsize=(14,5))
    librosa.display.waveshow(segment, sr=sampling_rate, color='blue')
    fig.suptitle(c)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa
import librosa.display
import os

DATA_DIR = '../data'

notHidden = lambda x: not x.startswith('.')
classes = filter(notHidden, os.listdir(DATA_DIR))

# Play the first sample of each class
for c in classes:
    wavs = filter(notHidden, os.listdir(DATA_DIR + '/' + c))
    w = next(wavs)
    path = os.path.join(DATA_DIR, c, w)
    sample, sampling_rate = librosa.load(path)
    
    sgram = librosa.stft(sample, n_fft=16384)
    sgram_mag, _ = librosa.magphase(sgram)
    mel_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sampling_rate)

    fig = plt.figure(figsize=(14,5))
    librosa.display.specshow(mel_sgram)
    fig.suptitle(c)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import os

DATA_DIR = '../data'

notHidden = lambda x: not x.startswith('.')
classes = filter(notHidden, os.listdir(DATA_DIR))

# Play the first sample of each class
for c in classes:
    wavs = filter(notHidden, os.listdir(DATA_DIR + '/' + c))
    w = next(wavs)
    path = os.path.join(DATA_DIR, c, w)
    sample, sampling_rate = librosa.load(path)
    segment = sample[:sampling_rate]    # Grab a 1s segment

    sgram = librosa.stft(segment, n_fft=16384)
    sgram_mag, _ = librosa.magphase(sgram)
    mel_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sampling_rate)

    fig = plt.figure(figsize=(14,5))
    librosa.display.specshow(mel_sgram)
    fig.suptitle(c)